# Play

In [202]:
def match(agents, render=False):
    obs, is_done = env.reset(), False    
    
    while not is_done:
        if agents[env.current_player] is None:
            state = env.get_state()
            prediction = calc_correct_output_sample(env.players[env.current_player].hand_cards, env.table_card, env.players[env.current_player].tricks, env.players[1 - env.current_player].tricks, env.players[1 - env.current_player].hand_cards)
           # print(prediction)
            env.set_state(state)
            #print("1", prediction, env.players[env.current_player].hand_cards[np.argmax(prediction)].color.name, env.players[env.current_player].hand_cards[np.argmax(prediction)].value.name)
            a = env.players[env.current_player].hand_cards[np.argmax(prediction)].id

            #break
        else:
            prediction = agents[env.current_player].predict_single(obs)[0]
           # print("2", prediction)
            #print(prediction, np.argmax(prediction))
            valid_moves = np.array([1 if card in env.players[env.current_player].hand_cards else 0 for card in env.cards])       

            prediction_valid = np.ma.masked_where(valid_moves == 0, prediction)
            a = np.argmax(prediction_valid)
            #print(a)
            
            
                    
        obs, rew, is_done, _ = env.step(a)
        
        if render:
            env.render('human')
            sleep(1)
       # if env.lastTrick  is not None:
        #    break
    return env.current_player

def compare(agent1, agent2):
    agents = [agent1, agent2]
    first_player_wins = 0
    
    for i in range(1000):
        start_player = random.randint(0, 1)
        winner = match([agents[start_player], agents[1 - start_player]])
        first_player_wins += ((winner == 0) == (start_player == 0))
        #print(start_player, winner)
    
    return first_player_wins / 1000